In [14]:
!pip install pandas nltk emoji
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
                                              0.0/22.8 MB ? eta -:--:--
                                              0.0/22.8 MB 1.4 MB/s eta 0:00:17
                                              0.1/22.8 MB 1.3 MB/s eta 0:00:18
                                              0.2/22.8 MB 1.6 MB/s eta 0:00:15
                                              0.3/22.8 MB 1.9 MB/s eta 0:00:13
     -                                        0.7/22.8 MB 2.8 MB/s eta 0:00:08
     -                                        1.0/22.8 MB 3.4 MB/s eta 0:00:07
     --                                       1.2/22.8 MB 3.7 MB/s eta 0:00:06
     --                                       1.7/22.8 MB 4.2 MB/s eta 0:00:05
     ---                                      2.1/22.8 MB 4.6 MB/s eta 0:00:05
     ----                                     2.4/22.8 MB 4.8 MB/s eta 0:00:05
     -----                                    3.0/22.8 MB 5.


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd

In [24]:
df_train = pd.read_excel('train1.xlsx')
df_test = pd.read_excel('test1.xlsx')

#### Image features

In [25]:
df_train['image_name'] = df_train['post_id'].astype(str) + '.jpg'
df_test['image_name'] = df_test['post_id'].astype(str) + '.jpg'

In [29]:
# Select numeric columns using the 'select_dtypes()' method
df_train_numeric = df_train.select_dtypes(include=['int64', 'float64'])
df_test_numeric = df_test.select_dtypes(include=['int64', 'float64'])

In [30]:
import pandas as pd
import numpy as np
import os
import skimage.io as io
import skimage.transform as sk_transform

In [52]:
def seqGenerator(X_train, y_train, batch_size, image_directory, df):
    index = 0
    num_train = X_train.shape[0]
    
    while True:
        images_batch = []
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            # Get the image name from the DataFrame
            image_name = df['image_name'].iloc[index]
            
            # Load the image from the local folder
            image_path = os.path.join(image_directory, image_name)
            img = io.imread(image_path)
            # Assuming you have defined IMAGE_SIZE and CHANNELS previously
            img = sk_transform.resize(img, (IMAGE_SIZE, IMAGE_SIZE, CHANNELS))
            img = img / 255.0  # Normalize the image
            
            images_batch.append(img)
            x_batch.append(X_train.iloc[index])  # Access the DataFrame correctly by using iloc
            y_batch.append(y_train.iloc[index])  # Access the DataFrame correctly by using iloc
            
            # Update the index and handle wrapping around the dataset
            index = (index + 1) % num_train
        
        images_batch = np.asarray(images_batch)
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        
        yield [images_batch, x_batch], y_batch

In [33]:
# Prepare the data
X_train, X_test = df_train_numeric.iloc[:, :-1], df_test_numeric.iloc[:, :-1]  # Features (all columns except the last one)
y_train, y_test = df_train_numeric.iloc[:, -1], df_test_numeric.iloc[:, -1]   # Target variable (last column)

In [35]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [48]:
temp_df=pd.concat([df_train,df_test])

In [49]:
temp_df

,username,num_followers,num_followings,post_id,day_of_week,hour,hashtags,mentions,caption,num_hashtags,num_mentions,hashtag_score,sentiment_score,num_likes,image_name
0,leanna_markoglou,23791,615,BTJVjFOAZRC,5,15,"makeup, hairstyle, youmagazine, photoshooting,...",NaN,My beauty friend.!! #makeup #hairstyle #youmag...,9,0,60654,1.6580,521,BTJVjFOAZRC.jpg
1,jls_by_dimitra,14001,1673,BSnouCMl9dx,6,13,NaN,NaN,"Πενταπλο βραχιολι ""χειροπεδα "" με μαγνητικό κο...",0,0,0,1.7633,401,BSnouCMl9dx.jpg
2,cookingnetwork,15040,3415,BStIhYmAa4Z,1,16,"puremagic, kayak_ice_cream, icecream, lemon, l...",NaN,Lemon Grass Ice Cream 🍋 #puremagic #kayak_ice_...,10,0,51521,1.0000,108,BStIhYmAa4Z.jpg
3,annamariabarouh,12761,42,BSzRv3gD9Yx,4,1,"easter, pastel, cupcakes, adorable, delicious",NaN,Easter is coming#Easter#pastel #cupcakes #ador...,5,0,37419,1.0000,250,BSzRv3gD9Yx.jpg
4,olgafarmaki,188519,997,BTPQT0Hl4pk,7,22,"clubc, reebokgreece, sneakers, sparkle, detail...",reebokgreece,Just the right amount of sparkle @reebokgreece...,7,1,25564,1.4215,5168,BTPQT0Hl4pk.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,twinsfashionn,17690,4121,BTNBi0cBapl,7,1,"twinsfashion, invernotwins, bonslooks, shopnow",NaN,Colete faux fur maravideuso 💗 SHOP NOW 📲www.tw...,4,0,0,1.0000,92,BTNBi0cBapl.jpg
3139,dorisliapata,12272,7305,BSvw4QVBkgT,2,17,"mycosmolook, eponymousview, instalifo",NaN,Είμαι έτοιμη για αλλαγές και είμαι σίγουρη ότι...,3,0,0,1.0000,1153,BSvw4QVBkgT.jpg
3140,oiko10,13458,3394,BSx-L0mFeXl,3,13,kai,NaN,#kai kapos etsi ta proina ksipnimata...apoktan...,1,0,8366,1.0000,8933,BSx-L0mFeXl.jpg
3141,giannis_kritikos,18106,4048,BTTS_yjjVqJ,2,12,NaN,NaN,Έτσι λοιπόν με ένα χαμόγελο ζει πάντα στις καρ...,0,0,0,1.0000,657,BTTS_yjjVqJ.jpg


In [57]:
import tensorflow as tf
from tensorflow_addons.metrics import RSquare

In [58]:
IMAGE_SIZE = 256
CHANNELS = 3

#Convolutional neural network for images
image_input = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS), name='image_input')
image_nn = Conv2D(4, (5,5), activation='relu')(image_input)
image_nn = Conv2D(16, (3,3), activation='relu')(image_input)
image_nn = MaxPooling2D(pool_size=(2,2))(image_nn)
image_nn = Conv2D(32, (3,3), activation='relu')(image_nn)
image_nn = Conv2D(64, (3,3), activation='relu')(image_nn)
image_nn = MaxPooling2D(pool_size=(2,2))(image_nn)
image_nn = Conv2D(128, (3,3), activation='relu')(image_nn)
image_nn = MaxPooling2D(pool_size=(2,2))(image_nn)
image_nn = Dropout(0.2)(image_nn)
image_nn = Conv2D(256, (3,3), activation='relu')(image_nn)
image_nn = MaxPooling2D(pool_size=(2,2))(image_nn)
image_nn = Dropout(0.2)(image_nn)
image_nn = Flatten()(image_nn)

#Neural network for user data and image metadata
data_input = Input(shape=(X_train.shape[1],), name='data_input')
data_nn = Dense(10, activation='relu')(data_input)

#Merge of the networks and last part of the network
output_nn = keras.layers.concatenate([image_nn, data_nn])
output_nn = Dense(500, activation='relu')(output_nn)
output_nn = Dropout(0.1)(output_nn)
output_nn = Dense(10, activation='relu')(output_nn)
output_nn = Dense(1, name='output')(output_nn)

#Model definition
model = Model(inputs=[image_input, data_input], outputs = [output_nn])

# Define custom metrics
mse = tf.keras.metrics.MeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError()
mape = tf.keras.metrics.MeanAbsolutePercentageError()

# Compile model with the custom metrics
model.compile(loss='mse', optimizer='rmsprop', metrics=[RSquare(), mse, mae, mape])

# Fit the model
trainingset_size = len(X_train)
batch_size = 8
losshistory = LossHistory()

model.fit_generator(seqGenerator(X_train, y_train, batch_size, 'All posts', temp_df), steps_per_epoch=trainingset_size // batch_size, epochs=25, callbacks=[losshistory])

C:\Users\User\AppData\Local\Temp\ipykernel_384\3056296733.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(seqGenerator(X_train, y_train, batch_size, 'All posts', temp_df), steps_per_epoch=trainingset_size // batch_size, epochs=25, callbacks=[losshistory])


Epoch 1/25
 400/1571 [======>.......................] - ETA: 1:05:42 - loss: 13600545.0000 - r_square: 0.4904 - mean_squared_error: 13600545.0000 - mean_absolute_error: 1602.7261 - mean_absolute_percentage_error: 227.6815

UnknownError: Graph execution error:

FileNotFoundError: No such file: 'f:\Needed stuff\Computer Science\Research stuff\Finale\All posts\BTBv56mlqR0.jpg'
Traceback (most recent call last):

  File "c:\Program Files\Python311\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Program Files\Python311\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Program Files\Python311\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\User\AppData\Local\Temp\ipykernel_384\35847